In [1]:
import pandas as pd
import re

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [3]:
cs_1 = pd.read_csv('../data/Caissabase1.csv')
cs_2 = pd.read_csv('../data/Caissabase2.csv')
cs_3 = pd.read_csv('../data/Caissabase3.csv')
cs_4 = pd.read_csv('../data/Caissabase4.csv')
cs_5 = pd.read_csv('../data/Caissabase5.csv')
cs_6 = pd.read_csv('../data/Caissabase6.csv')
cs_7 = pd.read_csv('../data/Caissabase7.csv')
cs_8 = pd.read_csv('../data/Caissabase8.csv')
cs_9 = pd.read_csv('../data/Caissabase9.csv')
cs_10 = pd.read_csv('../data/Caissabase10.csv')

In [4]:
caissabase = pd.concat([cs_1,cs_2,cs_3,cs_4,cs_5,cs_6,cs_7,cs_8,cs_9,cs_10])

In [5]:
caissabase.columns = ['col1','games']

In [6]:
caissabase = caissabase.drop(['col1'], axis = 1)

In [7]:
caissabase['event'] = caissabase['games'].str.extract('(\[Event[^\]]*\])')

In [8]:
caissabase['site'] = caissabase['games'].str.extract('(\[Site[^\]]*\])')

In [9]:
caissabase['date'] = caissabase['games'].str.extract('(\[Date[^\]]*\])')

In [10]:
caissabase['round'] = caissabase['games'].str.extract('(\[Round[^\]]*\])')

In [11]:
caissabase['white'] = caissabase['games'].str.extract('(\[White[^\]]*\])')

In [12]:
caissabase['black'] = caissabase['games'].str.extract('(\[Black[^\]]*\])')

In [13]:
caissabase['result'] = caissabase['games'].str.extract('(\[Result[^\]]*\])')

In [14]:
caissabase['black_elo'] = caissabase['games'].str.extract('(\[BlackElo[^\]]*\])')

In [15]:
caissabase['black_fide_id'] = caissabase['games'].str.extract('(\[BlackFideId[^\]]*\])')

In [16]:
caissabase['black_title'] = caissabase['games'].str.extract('(\[BlackTitle[^\]]*\])')

In [17]:
caissabase['eco'] = caissabase['games'].str.extract('(\[ECO[^\]]*\])')

In [18]:
caissabase['event_date'] = caissabase['games'].str.extract('(\[EventDate[^\]]*\])')

In [19]:
caissabase['opening'] = caissabase['games'].str.extract('(\[Opening[^\]]*\])')

In [20]:
caissabase['variation'] = caissabase['games'].str.extract('(\[Variation[^\]]*\])')

In [21]:
caissabase['white_elo'] = caissabase['games'].str.extract('(\[WhiteElo[^\]]*\])')

In [22]:
caissabase['white_fide_id'] = caissabase['games'].str.extract('(\[WhiteFideId[^\]]*\])')

In [23]:
caissabase['white_title'] = caissabase['games'].str.extract('(\[WhiteTitle[^\]]*\])')

In [24]:
caissabase['moves'] = caissabase['games'].str.extract('(\\n\\n[^%]*)')

In [25]:
caissabase = caissabase.rename(columns={'games':'full_pgn'})

In [26]:
caissabase['event'] = caissabase['event'].str.extract('"([^#]*)"')

In [27]:
caissabase['site'] = caissabase['site'].str.extract('"([^#]*)"')

In [28]:
caissabase['date'] = caissabase['date'].str.extract('"([^#]*)"')

In [29]:
caissabase['round'] = caissabase['round'].str.extract('"([^#]*)"')

In [30]:
caissabase['white'] = caissabase['white'].str.extract('"([^#]*)"')

In [31]:
caissabase['black'] = caissabase['black'].str.extract('"([^#]*)"')

In [32]:
caissabase['result'] = caissabase['result'].str.extract('"([^#]*)"')

In [33]:
caissabase['black_elo'] = caissabase['black_elo'].str.extract('"([^#]*)"')

In [34]:
caissabase['white_elo'] = caissabase['white_elo'].str.extract('"([^#]*)"')

In [35]:
caissabase['black_fide_id'] = caissabase['black_fide_id'].str.extract('"([^#]*)"')

In [36]:
caissabase['white_fide_id'] = caissabase['white_fide_id'].str.extract('"([^#]*)"')

In [37]:
caissabase['black_title'] = caissabase['black_title'].str.extract('"([^#]*)"')

In [38]:
caissabase['white_title'] = caissabase['white_title'].str.extract('"([^#]*)"')

In [39]:
caissabase['eco'] = caissabase['eco'].str.extract('"([^#]*)"')

In [40]:
caissabase['event_date'] = caissabase['event_date'].str.extract('"([^#]*)"')

In [41]:
caissabase['opening'] = caissabase['opening'].str.extract('"([^#]*)"')

In [42]:
caissabase['variation'] = caissabase['variation'].str.extract('"([^#]*)"')

In [43]:
caissabase['moves'] = caissabase['moves'].str.replace(r'1-0', r'')
caissabase['moves'] = caissabase['moves'].str.replace(r'0-1', r'')
caissabase['moves'] = caissabase['moves'].str.replace(r'1/2-1/2', r'')

In [44]:
caissabase['@'] = '@'

In [45]:
caissabase['moves'] = caissabase['moves'] + caissabase['@']

In [46]:
for x in range (1,21):
    y = ' ' + str(x + 1)
    x = str(x)
    caissabase[x] = caissabase['moves'].str.extract('('+x+'\.[^@.]*)')
    caissabase[x] = caissabase[x].str.replace(y, '')
    caissabase[x] = caissabase[x].str.replace(x + '\.', '')

<ipython-input-46-336d59ff31a9>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  caissabase[x] = caissabase[x].str.replace(x + '\.', '')


In [53]:
caissabase = caissabase.drop(['full_pgn', 'moves'], axis = 1)

In [56]:
for x in range (1,21):
    x = str(x)
    w = x + 'w'
    b = x + 'b'
    caissabase[x] = caissabase[x].str.strip()
    caissabase[[w, b]] = caissabase[x].str.split(' ', n = 1, expand = True)

In [61]:
caissabase = caissabase.drop(['@','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'], axis = 1)

In [62]:
caissabase.to_csv('../data/caissabase_df.csv')

In [49]:
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    print(caissabase['0'].value_counts(dropna = False))